In [1]:
%pip install gensim
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import numpy as np

In [3]:
import gensim.downloader as api
wv=api.load('word2vec-google-news-300')
vec_king=wv['king']

In [4]:
import pandas as pd
messages=pd.read_csv('SMSSpamCollection.txt',sep='\t',names=["labels","message"])

In [5]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()


In [26]:
import re
import nltk
nltk.download('wordnet', download_dir='c:/Users/ASUS/anaconda3/envs/myenv/nltk_data')
nltk.download('omw-1.4', download_dir='c:/Users/ASUS/anaconda3/envs/myenv/nltk_data')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
import sklearn



[nltk_data] Downloading package wordnet to
[nltk_data]     c:/Users/ASUS/anaconda3/envs/myenv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     c:/Users/ASUS/anaconda3/envs/myenv/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\ASUS\anaconda3\envs\myenv\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\ASUS\anaconda3\envs\myenv\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\ASUS\anaconda3\envs\myenv\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()

    review=[lemmatizer.lemmatize(word) for word in review]
    review=' '.join(review)
    corpus.append(review)

In [8]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [9]:
model=gensim.models.Word2Vec(words)
model.wv.index_to_key

['to',
 'you',
 'the',
 'and',
 'it',
 'in',
 'is',
 'me',
 'my',
 'for',
 'your',
 'call',
 'of',
 'that',
 'have',
 'on',
 'now',
 'are',
 'can',
 'so',
 'but',
 'not',
 'or',
 'we',
 'do',
 'get',
 'at',
 'ur',
 'if',
 'will',
 'be',
 'with',
 'no',
 'just',
 'this',
 'gt',
 'lt',
 'go',
 'how',
 'up',
 'when',
 'day',
 'ok',
 'what',
 'free',
 'from',
 'all',
 'out',
 'know',
 'll',
 'come',
 'like',
 'good',
 'time',
 'am',
 'then',
 'got',
 'wa',
 'there',
 'he',
 'love',
 'text',
 'only',
 'want',
 'send',
 'one',
 'need',
 'txt',
 'today',
 'by',
 'going',
 'don',
 'stop',
 'she',
 'home',
 'about',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'dont',
 'our',
 'think',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'they',
 'later',
 'please',
 'any',
 'pls',
 'her',
 'ha',
 'did',
 'co',
 'msg',
 'been',
 'min',
 'an',
 'some',
 'dear',
 'night',
 'make',
 'who',
 'here',
 'message',
 'well',
 'say',
 'where',
 're',
 'thing',
 'much',
 'hope

In [10]:
model.wv['good'].shape

(100,)

In [11]:
def avg_word2vec(docs):
    vectors = [model.wv[word] for word in docs if word in model.wv.index_to_key]
    if len(vectors) == 0:                     # avoid mean([]) error
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


In [12]:
! pip install tqdm

In [13]:
from tqdm import tqdm

In [14]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 5569/5569 [00:02<00:00, 2759.71it/s]


In [15]:
## independent feature 
X_new=np.array(X)

In [32]:
## output feature
y=messages[list(map(lambda x: len(x)>0, corpus))]
y=pd.get_dummies(y['labels'])
y=y.iloc[:,0].values

In [33]:
y.shape

(5569,)

In [19]:
df=pd.DataFrame()
df = pd.DataFrame(np.vstack(X))

In [22]:
df.shape

(5569, 100)

In [36]:
df['Output']=y

In [37]:
df.dropna(inplace=True)

In [38]:
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [60]:
X=df.drop(df['Output'],inplace=True)


In [41]:
## Train test split 
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
    

In [61]:
y_train

889     1
547     1
2792    1
348     1
1075    1
       ..
5040    0
4127    1
318     1
678     1
519     1
Name: Output, Length: 4176, dtype: int64

In [42]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [62]:
classifier.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [63]:
y_pred=classifier.predict(X_test)

In [65]:
from sklearn.metrics import accuracy_score,classification_report
accuracy=accuracy_score(y_test,y_pred)
report=classification_report(y_test,y_pred)
print(report)
print(accuracy)

              precision    recall  f1-score   support

           0       0.90      0.84      0.87       202
           1       0.97      0.98      0.98      1191

    accuracy                           0.96      1393
   macro avg       0.94      0.91      0.93      1393
weighted avg       0.96      0.96      0.96      1393

0.9641062455132807
